## Goals
 - Take the merged predictions and evaluate the prediction accuracy using the 2 different approaches
 1. Look at the anaphora tags and then cross-reference co-reference labels
 2. Use the co-reference chains directly

In [28]:
import dill
import pandas as pd

from Settings import Settings
from collections import defaultdict
from BrattEssay import ANAPHORA

from results_common import get_essays, validate_essays, tally_essay_attributes
from process_essays_coref import get_coref_processed_essays
from metrics import get_metrics_raw

# progress bar widget
from ipywidgets import IntProgress
from IPython.display import display

DATASET = "CoralBleaching" # CoralBleaching | SkinCancer

settings = Settings()
root_folder = settings.data_directory + DATASET + "/Thesis_Dataset/"
stanford_coref_predictions_folder = root_folder + "CoReference/"
print("CoRef Data: ", stanford_coref_predictions_folder)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
CoRef Data:  /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/CoReference/


In [2]:
training_essays = get_essays(stanford_coref_predictions_folder, "Training")

Found file /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/CoReference/training_processed.dill


In [7]:
test_essays = get_essays(stanford_coref_predictions_folder, "Test")

Found file /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/CoReference/test_processed.dill


In [9]:
all_essays = training_essays + test_essays

### Validate the Lengths

In [8]:
validate_essays(training_essays)
validate_essays(test_essays)

Essays validated
Essays validated


In [11]:
# ner_tally = tally_essay_attributes(all_essays, attribute_name="pred_ner_tags_sentences")
pos_tally = tally_essay_attributes(all_essays, attribute_name="pred_pos_tags_sentences")

## Look at the Anaphor Tags

In [13]:
from results_procesor import is_a_regular_code

cc_tally = defaultdict(int)
cr_tally = defaultdict(int)
reg_tally = defaultdict(int)
for e in all_essays:
    for sent in e.sentences:
        for wd, tags in sent:
            for t in tags:
                if is_a_regular_code(t):
                    reg_tally[t] += 1
                if ANAPHORA in t and "other" not in t:
                    if "->" in t:
                        cr_tally[t] += 1
                    elif "Anaphor:[" in t:
                        cc_tally[t] += 1

reg_tags = sorted(reg_tally.keys())
all_ana_tags = sorted(cc_tally.keys())
assert len(reg_tags) == len(all_ana_tags)
all_ana_tags

['Anaphor:[11]',
 'Anaphor:[12]',
 'Anaphor:[13]',
 'Anaphor:[14]',
 'Anaphor:[1]',
 'Anaphor:[2]',
 'Anaphor:[3]',
 'Anaphor:[4]',
 'Anaphor:[50]',
 'Anaphor:[5]',
 'Anaphor:[5b]',
 'Anaphor:[6]',
 'Anaphor:[7]']

## Prepare POS Tag Filters

In [14]:
pos_nouns = set([pos for pos in pos_tally.keys() if pos.strip()[:2] == "NN"])
pos_verbs = set([pos for pos in pos_tally.keys() if pos.strip()[:2] == "VB"])
pos_pronouns = {"PRP","PRP$", "WP", "WP$"}
pos_determiners = {"DT","WDT","PDT"} # the, a, which, that, etc
pos_pron_dt = pos_pronouns | pos_determiners
# for meaning of pen treebank tags - https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
pos_nouns, pos_verbs, pos_pronouns, pos_determiners, pos_pron_dt

({'NN', 'NNP', 'NNPS', 'NNS'},
 {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'},
 {'IN', 'NN', 'NNP', 'NNPS', 'NNS', 'PRP', 'PRP$', 'WP', 'WP$'})

In [15]:
dict_pos_filter = {
            "None": None,
            "PRN": pos_pronouns,
            "DT": pos_determiners,
            "PRN+DT": pos_pron_dt
}

dict_pos_ch_filter = {
    "None": None,
    "NN": pos_nouns,
    "VB": pos_verbs
}

In [16]:
def blank_if_none(val):
    return "-" if (val is None or not val or str(val).lower() == "none") else val

def process_sort_results(df_results):
    df_disp = df_results[["f1_score","precision","recall", LOOK_BACK, MAX_PHRASE, POS_FLTR, POS_CHAIN_FLTR]]
    return df_disp.sort_values("f1_score", ascending=False)

In [17]:
phrase_len = [None] + list(range(1,11))
look_back_vals = [True,False]

In [22]:
LOOK_BACK = "Look back"
MAX_PHRASE = "Max phrase"
POS_FLTR = "POS filter"
POS_CHAIN_FLTR = "Pos chain filter"

# Grid Search With Anaphora Prediction Filters

### Training

In [47]:
def grid_search(essays, format_ana_tags, filter_to_predicted_tags, expected_tags):

    # set up progress bar
    max_count = len(look_back_vals)  * len(phrase_len) * len(dict_pos_filter) * len(dict_pos_ch_filter)
    iprogress_bar = IntProgress(min=0, max=max_count) # instantiate the bar
    display(iprogress_bar) # display the bar

    rows_ana = []
    for look_back_only in look_back_vals:
        for pos_key, pos_filter in dict_pos_filter.items():
            for pos_ch_key, pos_ch_filter in dict_pos_ch_filter.items():                
                for max_cref_phrase_len in phrase_len:
                    proc_essays = get_coref_processed_essays(
                        essays=essays, format_ana_tags=format_ana_tags, 
                        filter_to_predicted_tags=filter_to_predicted_tags, look_back_only=look_back_only,
                        max_cref_phrase_len=max_cref_phrase_len, ner_ch_filter=None, 
                        pos_filter=pos_filter, pos_ch_filter=pos_ch_filter)
                    metrics = get_metrics_raw(proc_essays, expected_tags=expected_tags,  micro_only=True)
                    row = metrics["MICRO_F1"]
                    row[LOOK_BACK] = look_back_only
                    row[MAX_PHRASE] = blank_if_none(max_cref_phrase_len)
                    row[POS_FLTR] = blank_if_none(pos_key)
                    row[POS_CHAIN_FLTR] = blank_if_none(pos_ch_key)
                    rows_ana.append(row)
                    iprogress_bar.value += 1

    df_results = pd.DataFrame(rows_ana)
    return df_results

In [48]:
filter_to_predicted_tags = True
format_ana_tags=True # Format tags with Anaphora[xyz]

df_train_raw = grid_search(essays=training_essays, 
                           expected_tags=all_ana_tags,
                           filter_to_predicted_tags=filter_to_predicted_tags, format_ana_tags=format_ana_tags)

IntProgress(value=0, max=264)

In [49]:
process_sort_results(df_train_raw).head()

,f1_score,precision,recall,Look back,Max phrase,POS filter,Pos chain filter
0,0.037534,0.241379,0.020349,True,-,-,-
10,0.037534,0.241379,0.020349,True,10,-,-
21,0.037534,0.241379,0.020349,True,10,-,NN
20,0.037534,0.241379,0.020349,True,9,-,NN
109,0.037534,0.241379,0.020349,True,10,PRN+DT,-


### Test

In [61]:
filter_to_predicted_tags = True
format_ana_tags=True # Format tags with Anaphora[xyz]

df_test_raw = grid_search(essays=test_essays, 
                          expected_tags=all_ana_tags,
                           filter_to_predicted_tags=filter_to_predicted_tags, format_ana_tags=format_ana_tags)

IntProgress(value=0, max=264)

In [62]:
process_sort_results(df_test_raw).head()

,f1_score,precision,recall,Look back,Max phrase,POS filter,Pos chain filter
263,0.05,1.0,0.025641,False,10,PRN+DT,VB
154,0.05,1.0,0.025641,False,-,-,VB
22,0.05,1.0,0.025641,True,-,-,VB
130,0.05,1.0,0.025641,True,9,PRN+DT,VB
66,0.05,1.0,0.025641,True,-,DT,-


## Grid Search without Anaphora Predictions

### Training

In [59]:
filter_to_predicted_tags = False
format_ana_tags=True # Format tags with Anaphora[xyz]

df_train_raw_cref = grid_search(essays=training_essays, 
                                expected_tags=all_ana_tags,
                           filter_to_predicted_tags=filter_to_predicted_tags, format_ana_tags=format_ana_tags)

IntProgress(value=0, max=264)

In [60]:
process_sort_results(df_train_raw_cref).head()

,f1_score,precision,recall,Look back,Max phrase,POS filter,Pos chain filter
119,0.024419,0.014123,0.090116,True,9,PRN+DT,NN
120,0.024257,0.014014,0.090116,True,10,PRN+DT,NN
110,0.024151,0.013877,0.093023,True,-,PRN+DT,NN
108,0.024035,0.013739,0.095930,True,9,PRN+DT,-
109,0.023878,0.013636,0.095930,True,10,PRN+DT,-


### Test

In [57]:
filter_to_predicted_tags = False
format_ana_tags=True # Format tags with Anaphora[xyz]

df_test_raw_cref = grid_search(essays=test_essays, 
                               expected_tags=all_ana_tags,
                           filter_to_predicted_tags=filter_to_predicted_tags, format_ana_tags=format_ana_tags)

IntProgress(value=0, max=264)

In [58]:
process_sort_results(df_test_raw_cref).head()

,f1_score,precision,recall,Look back,Max phrase,POS filter,Pos chain filter
97,0.041667,0.111111,0.025641,True,9,DT,VB
96,0.041667,0.111111,0.025641,True,8,DT,VB
129,0.040000,0.090909,0.025641,True,8,PRN+DT,VB
130,0.039216,0.083333,0.025641,True,9,PRN+DT,VB
98,0.038462,0.076923,0.025641,True,10,DT,VB
